### Deliverable 1: Preprocessing the Data for a Neural Network

In [68]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd 

# Import and read the charity_data.csv.
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [90]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.head()

KeyError: "['EIN' 'NAME'] not found in axis"

In [91]:
# Determine the number of unique values in each column.
application_df.nunique()

STATUS                                                               2
ASK_AMT                                                           8747
IS_SUCCESSFUL                                                        2
NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC       2
NAME_100 BLACK MEN OF AMERICA                                        2
                                                                  ... 
INCOME_AMT_25000-99999                                               2
INCOME_AMT_50M+                                                      2
INCOME_AMT_5M-10M                                                    2
SPECIAL_CONSIDERATIONS_N                                             2
SPECIAL_CONSIDERATIONS_Y                                             2
Length: 19612, dtype: int64

In [92]:
# Look at APPLICATION_TYPE value counts for binning
AppCounts = application_df['APPLICATION_TYPE'].value_counts()
AppCounts

KeyError: 'APPLICATION_TYPE'

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
AppCounts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
# setting at 500 since I saw t10 at 528
applicationBin_other = AppCounts[AppCounts < 500].index.tolist()

# Replace in dataframe
for app in applicationBin_other:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
    
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
ClassCounts = application_df['CLASSIFICATION'].value_counts()
ClassCounts

In [ ]:
# Visualize the value counts of CLASSIFICATION
ClassCounts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..?
# setting at 1000 since I saw c2100 at 1883
classBin_other = ClassCounts[ClassCounts < 1000].index.tolist()

# Replace in dataframe
for cls in classBin_other:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
# create new df before merging later
encoded_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encoded_df.columns = enc.get_feature_names(application_cat)
encoded_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encoded_df, left_index=True, right_index=True)
application_df = application_df.drop(application_cat, 1)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# using multi-layer perceptron (two layers)
numInputFeatures = len(X_train[0])

# layer1 = input layer, typically equals number of input variables in data
layer1 = 15
# layer 2 = hidden layer, typically 2/3 of input layer
layer2 = 10
# layer 3 = hidden layer
layer3= 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1, 
                          input_dim=numInputFeatures, 
                          activation="relu")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, 
                             activation="relu"))

# adding a third layer to increase accuracy 
nn.add(tf.keras.layers.Dense(units=layer3, 
                             activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    # checkpoint directory and file structure defined above
    filepath=checkpoint_path,
    # notified when checkpoint is being saved to the directory
    verbose=1,
    # checkpoint files take small space
    save_weights_only=True,
    # checkpoints saved every epoch
    save_freq='epoch')

In [ ]:
# Train the model
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [19]:
# Export the model to HDF5 file
#nn.save("AlphabetSoupCharity_optimization.h5")